In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data load and viewing

In [ ]:
train_data= pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv", index_col="row_id")

duplicates_train = train_data.duplicated().sum()
print('Duplicates in train data: {0}'.format(duplicates_train))

train_data.drop_duplicates(keep='first', inplace=True)
duplicates_train = train_data.duplicated().sum()

print('Train data shape:', train_data.shape)
print('Duplicates in train data: {0}'.format(duplicates_train))

In [ ]:
sample_submission=pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv")
# train=pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/train.csv")
test=pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/test.csv")

**Train data**

In [ ]:
# train.head()

In [ ]:
# train.describe()

In [ ]:
train_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
# print(f'Number of rows in train data: {train.shape[0]}')
# print(f'Number of columns in train data: {train.shape[1]}')
# print(f'Number of values in train data: {train.count().sum()}')
# print(f'\033[91mNumber missing values in train data: {sum(train.isna().sum())}')
# print(f'\033[91mNumber of duplicates in train data: {train.duplicated().sum()}')

In [ ]:
# print(sorted(train.columns.unique()))
# print(train['target'].unique())
print(sorted(train_data.columns.unique()))
print(train_data['target'].unique())

**Test data**

In [ ]:
test.head()

In [ ]:
test.describe()

In [ ]:
print(f'Number of rows in test data: {test.shape[0]}')
print(f'Number of columns in test data: {test.shape[1]}')
print(f'Number of values in test data: {test.count().sum()}')
print(f'\033[91mNumber missing values in test data: {sum(test.isna().sum())}')
print(f'\033[91mNumber of duplicates in test data: {test.duplicated().sum()}')

In [ ]:
print(sorted(test.columns.unique()))

**Sample Submission**

In [ ]:
sample_submission.head()

# Data cleanup and sample weight

# Variable assignment and column drop

In [ ]:
# trainX = train.drop(columns=['row_id', 'target'])
# trainY = train[['target']]
trainX = train_data.drop(columns=['target'])
trainY = train_data[['target']]
testX = test.drop(columns=['row_id'])
testY = test[['row_id']]

In [ ]:
trainX.head()

# Model = RandomForestClassifier

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
trainY = encoder.fit_transform(trainY)

In [ ]:
import seaborn as sns
from sklearn.ensemble import ExtraTreesClassifier

#check max_depth and other parameters
model = ExtraTreesClassifier(n_estimators=3000, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features='auto', 
                               max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=False, n_jobs=-1, random_state=1000, verbose=1, max_samples=1)
model.fit(trainX, trainY)

y_pred = model.predict(testX)

In [ ]:
# from sklearn.neighbors import RadiusNeighborsClassifier
# model = RadiusNeighborsClassifier(radius=18, weights='distance',
#                                 p=1, outlier_label=-1, n_jobs=-1)
# model.fit(trainX, trainY)
# y_pred = model.predict(testX)
# print('Unique predictions:', np.unique(y_pred))
# print('Frequencies:', np.unique(y_pred, return_counts=True)[1])
# print('Samples:', len(y_pred))
# print('Predicted samples:', 
#       len(y_pred) - np.unique(y_pred, return_counts=True)[1][0])

# Submission

In [ ]:
y_pred = encoder.inverse_transform(y_pred)
submission = pd.DataFrame({'row_id': test.row_id, 'target': y_pred})
submission.to_csv('submission.csv', index=False)
submission.head()

# Visualization and accuracy

In [ ]:
trainY.shape, y_pred.shape

In [ ]:
type(trainY)

In [ ]:
accX = pd.DataFrame(trainY)
accX.head()

In [ ]:
accY = pd.DataFrame(y_pred)
accY.head()

In [ ]:
accX = encoder.fit_transform(accX)
accY = encoder.fit_transform(accY)

In [ ]:
accX.shape, accY.shape

In [ ]:
accX = np.reshape(accX, (100000, 1))
accY = np.reshape(accY, (100000, 1))

In [ ]:
accX.shape, accY.shape

In [ ]:
from sklearn.metrics import accuracy_score

# model.score(accX, accY)
print(f"{accuracy_score(accX, accY):.2%}\n")

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()

# Model (can also use single decision tree)
# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier(n_estimators=10)

# Train
model.fit(iris.data, iris.target)
# Extract single tree
estimator = model.estimators_[5]

from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = iris.feature_names,
                class_names = iris.target_names,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import RocCurveDisplay
from sklearn.datasets import load_wine

svc_disp = RocCurveDisplay.from_estimator(model, testX, testY)
ax = plt.gca()
rfc_disp = RocCurveDisplay.from_estimator(model, testX, testY, ax=ax, alpha=0.8)
svc_disp.plot(ax=ax, alpha=0.8)